In [1]:
# Test ML compute
import json
import os
from pathlib import Path
from tempfile import NamedTemporaryFile
from threading import local

import json
import cloudpathlib
import pandas as pd
import promptflow as pf
from azure.ai.ml.entities import Data
from azure.core.exceptions import ResourceNotFoundError
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from jinja2 import Environment, FileSystemLoader
from promptflow.azure import PFClient as AzurePFClient
from promptflow.client import PFClient as LocalPFClient
from promptflow.core import AzureOpenAIModelConfiguration, Prompty
from promptflow.tracing import start_trace, trace

from datatools.chains.parser import parse_json
from datatools.chains.prompts import STANDARDS
from datatools.gcloud import GCloud
from datatools.chains.toxicity import TOXCLIENTS

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, load_component, Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

/opt/conda/envs/python311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not load partial prompt HATESPEECH.examples_disabled from /workspaces/dev/datatools/chains/templates/hatespeech.yaml: 'str' object does not support item assignment e.args=("'str' object does not support item assignment",)


In [29]:
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import (
    ClientAuthenticationError,
    HttpResponseError,
    ResourceNotFoundError,
)
from azure.identity import DefaultAzureCredential
from cloudpathlib import CloudPath
from jinja2 import Environment, FileSystemLoader, Template
from promptflow._sdk.entities._flows.flex import FlexFlow
from promptflow.azure import PFClient as AzurePFClient
from promptflow.client import PFClient as LocalPFClient
from promptflow.core import (
    AzureOpenAIModelConfiguration,
    OpenAIModelConfiguration,
    Prompty,
)
from promptflow.core._prompty_utils import convert_prompt_template
from promptflow.entities import AzureOpenAIConnection, FlowContext, CustomConnection, OpenAIConnection
from promptflow.tracing import start_trace, trace
from sympy import false
from anthropic import APIConnectionError as AnthropicAPIConnectionError
from anthropic import RateLimitError as AnthropicRateLimitError
from google.api_core.exceptions import ResourceExhausted
from google.generativeai.types.generation_types import (
    BlockedPromptException,
    StopCandidateException,
)
from langchain_core.messages import BaseMessage
from langchain_core.runnables import ConfigurableField, Runnable
from openai import APIConnectionError as OpenAIAPIConnectionError
from openai import RateLimitError as OpenAIRateLimitError
from tenacity import (
    retry,
    retry_if_exception_type,
    stop_after_attempt,
    wait_exponential_jitter,
    wait_random_exponential,
)
import urllib3
import requests

from datatools.exceptions import RateLimit
from datatools.utils import TooManyRequests
from datatools.json_utils import parse_json

In [5]:
from datatools.azcloud import auth
credential = auth()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)


Found the config file in: ./.azureml/config.json


In [6]:
data_input = Input(
    path="azureml:vaw_train:2",
)
pipeline_output = Output(
    # Provide custom flow output file path if needed
    # path="azureml://datastores/<data_store_name>/paths/<path>",
    type=AssetTypes.URI_FOLDER,
    # rw_mount is suggested for flow output
    mode="rw_mount",
)
pipeline_output

In [19]:
DATASET = 'gs://dmrc-platforms/data/osb_drag_toxic_train.jsonl'
with NamedTemporaryFile(delete=False, suffix='.jsonl', mode='w') as f:
    dataset = f.name
cloudpathlib.CloudPath(DATASET).download_to(dataset)
print(dataset)

/tmp/tmpihmurvfh.jsonl


In [48]:
template_path='apply_rules.prompty'
standards_path='common/criteria_gelber.jinja2'
instructions_path='common/instructions.jinja2'
process_path='common/process.jinja2'
content = None

#connection=azclient.ml_client.serverless_endpoints.get(name="llama31_8b")
localclient = LocalPFClient()
connection=localclient.connections.get(name="llama31_8b")

model_config={
    "apply_rules": {
        "connection": connection,
        "model": "llama31_8b",
    }
}
from flows.automod.judge.judge import Judger
flow = Judger(standards_path=standards_path, template_path=template_path, model_config=model_config, instructions_path=instructions_path, process_path=process_path)

# run the flow as function, which will be recorded in the trace
result = flow(content="Kill all men")

2024-08-10 11:20:49 6c4962fcd0f9 azure.identity._credentials.environment environment.py[ 115] INFO No environment configuration found.
2024-08-10 11:20:49 6c4962fcd0f9 azure.identity._credentials.managed_identity managed_identity.py[  96] INFO ManagedIdentityCredential will use IMDS
2024-08-10 11:20:49 6c4962fcd0f9 azure.core.pipeline.policies.http_logging_policy _universal.py[ 513] INFO Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.18.0b1 Python/3.11.0 (Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.31)'
No body was attached to the request
2024-08-10 11:20:50 6c4962fcd0f9 azure.identity._credentials.chained chained.py[  95] INFO DefaultAzureCredential acquired a token from AzureCliCredential
2024-08-10 11:20:52 6c4962fcd0f9 azure.identity._credentials.environment environment.py[ 115] INFO No environment configuration found.
2024-0

2024-08-10 11:20:52 6c4962fcd0f9 azure.core.pipeline.policies.http_logging_policy _universal.py[ 510] INFO Request URL: 'https://eastus-8.in.applicationinsights.azure.com/v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '2258'
    'Accept': 'application/json'
    'x-ms-client-request-id': '9a6e6a58-570a-11ef-a7f3-0242ac110002'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.0 (Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.31)'
A body is sent with the request
2024-08-10 11:20:53 6c4962fcd0f9 azure.core.pipeline.policies.http_logging_policy _universal.py[ 549] INFO Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'Microsoft-HTTPAPI/2.0'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Sat, 10 Aug 2024 11:20:52 GMT'
2024-08-10 11:20:53 6c4962fcd0f9 a

[2024-08-10 11:20:54 +0000][promptflow.core._prompty_utils][ERROR] - Exception occurs: AuthenticationError: Error code: 401 - {'error': {'code': 'PermissionDenied', 'message': 'Principal does not have access to API/Operation.'}}


WrappedOpenAIError: OpenAI API hits AuthenticationError: Principal does not have access to API/Operation. If you are using azure openai connection, please make sure you have proper role assignment on your azure openai resource. You can refer to https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/role-based-access-control

In [8]:
FLOW_DIR = "automod/flows/automod/judge"

flow_component = load_component("automod/flows/automod/judge/flow.dag.yaml")
x = flow_component(data=data_input, text="${data.text}", alt_text="${data.alt_text}", record_id="${data.record_id}", run_local_models=False, connections=connections)


x = localclient.run(flow=FLOW_DIR, data=dataset, connections=connections, column_mapping=columns, stream=False)

In [23]:
columns = dict(source="${data.source}", record_id="${data.id}",text="${data.text}",
                       alt_text="${data.alt_text}", id="${data.id}", datasets=["drag queens"], use_alt_text=True, run_local_models=False)



In [43]:
connection = localclient.connections.get(name="llama31_8b")
connection

In [47]:
flow = FlexFlow.load(FLOW_DIR, model_config=connection)

flow(text="hello")

[2024-08-10 09:49:40 +0000][flowinvoker][INFO] - Getting connections from pf client with provider from args: azureml://subscriptions/7e7e056a-4224-4e26-99d2-1e3f9a688c50/resourcegroups/rg-suzor_ai/providers/Microsoft.MachineLearningServices/workspaces/automod...
[2024-08-10 09:49:40 +0000][flowinvoker][INFO] - Promptflow get connections successfully. keys: dict_keys([])
[2024-08-10 09:49:40 +0000][flowinvoker][INFO] - Promptflow executor starts initializing...


ResolveToolError: Tool load failed in 'apply_rules': (GetConnectionError) Get connection 'None' for node 'apply_rules' error: Connection None not found in dict connection provider. Available keys are [].

In [45]:
connections={
        "apply_rules": {
            "connection": connection,
            "model": "llama31_8b",
        }
        }
conn=azclient.ml_client.serverless_endpoints.get(name="Meta-Llama-31-70B-Instruct-jekeg")
deployment = pf.core._connection.ServerlessConnection(api_base='https://Meta-Llama-31-70B-Instruct-jekeg.eastus.models.ai.azure.com/v1/chat/completions', api_key='dUVZu1EWbMPI1bBJJbNthv9wt4Q2dO6u')
model_config =
override_model = {
    "configuration": {
        "api_key": self.model_config['configuration'].get('api_key'),
        "base_url": self.model_config['configuration'].get('base_url'),
    },
}
x.connections=connections

RepresenterError: cannot represent an object: ConnectionFields.CONNECTION

In [39]:
x=flow.context

In [40]:
x.connections

{}

[2024-08-10 09:42:16 +0000][flowinvoker][INFO] - Getting connections from pf client with provider from args: azureml://subscriptions/7e7e056a-4224-4e26-99d2-1e3f9a688c50/resourcegroups/rg-suzor_ai/providers/Microsoft.MachineLearningServices/workspaces/automod...
[2024-08-10 09:42:16 +0000][flowinvoker][INFO] - Promptflow get connections successfully. keys: dict_keys([])
[2024-08-10 09:42:16 +0000][flowinvoker][INFO] - Promptflow executor starts initializing...


ResolveToolError: Tool load failed in 'apply_rules': (GetConnectionError) Get connection 'None' for node 'apply_rules' error: Connection None not found in dict connection provider. Available keys are [].